In [6]:
import duckdb
import pandas as pd

con = duckdb.connect("dev_database.duckdb")
con.sql("USE dev_database.dev_sode")
con.sql("SHOW TABLES;").show()
con.close()

┌─────────┐
│  name   │
│ varchar │
├─────────┤
│ states  │
└─────────┘



In [11]:
df = con.sql("select * from fact_daily_climate_parameters ").df()

In [12]:
df.head()

,reading_date,source_flag,measurement_flag,PRCP_avg,PRCP_max,PRCP_min,TMAX_min,TMAX_max,TMAX_avg,TMIN_min,TMIN_max,TMIN_avg
0,2023-01-08,7,None,18.340815,914.0,0.0,-206.0,356.0,66.392519,-317.0,278.0,-39.896693
1,2023-01-08,T,None,37.483213,508.0,0.0,-228.0,123.0,12.000000,-344.0,66.0,-67.767470
2,2023-01-08,S,B,6.659341,180.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-01-08,N,T,0.000000,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-01-09,S,B,12.634538,559.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


## Stations

In [32]:
import boto3
import pandas as pd
import re
bucket_name = 'noaa-ghcn-pds'
key = 'ghcnd-stations.txt'

client = boto3.client('s3')

text_obj = client.get_object(Bucket=bucket_name, Key=key)
text_body = text_obj['Body'].read().decode('utf-8')
#text_csv = re.sub(r'\s+',',',text_body)

# text_body.split('\n')
with open('stations.txt','w') as f:
    f.write(text_body)




In [35]:
df_stations = pd.read_fwf('stations.txt')
df_stations.columns=['station_id','latitude','longitude','state','station_name','gsn_flag','hcn_crn_flag','wmo_number']
df_stations.to_csv('stations.csv')

## Countries

In [31]:
import boto3
import pandas as pd
import re
bucket_name = 'noaa-ghcn-pds'
key = 'ghcnd-countries.txt'

client = boto3.client('s3')

text_obj = client.get_object(Bucket=bucket_name, Key=key)
text_body = text_obj['Body'].read().decode('utf-8')
#text_csv = re.sub(r'\s+',',',text_body)

# text_body.split('\n')
with open('countries.txt','w') as f:
    f.write(text_body)




In [34]:
df_countries = pd.read_fwf('countries.txt')
df_countries.columns=['country_code','country_name']
df_countries.to_csv('countries.csv')

# States

In [36]:
import boto3
import pandas as pd
import re
bucket_name = 'noaa-ghcn-pds'
key = 'ghcnd-states.txt'

client = boto3.client('s3')

text_obj = client.get_object(Bucket=bucket_name, Key=key)
text_body = text_obj['Body'].read().decode('utf-8')
#text_csv = re.sub(r'\s+',',',text_body)

# text_body.split('\n')
with open('state.txt','w') as f:
    f.write(text_body)

df_states = pd.read_fwf('state.txt')
df_states.columns=['state_code','state_name']
df_states.to_csv('states.csv')

In [4]:
# df_stations= df.join(df_countries,on=)
def push_metadata(data, database_name, table_name):

    conn = duckdb.connect(database=database_name,read_only=False)
    conn.execute("CREATE SCHEMA IF NOT EXISTS dev_sode;")
    df = pd.read_csv(data)

    conn.execute(f"CREATE TABLE IF NOT EXISTS {table_name} AS SELECT * FROM df")

    conn.close()

In [9]:
push_metadata('states.csv', "dev_database.duckdb", "states")
push_metadata('stations.csv',"dev_database.duckdb", "stations")
push_metadata('countries.csv', "dev_database.duckdb", "countries")